<a href="https://colab.research.google.com/github/githubforkj/signate/blob/main/mufj/02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# フォーラムで提示された強プレイヤーの写経

(完全理解も同時に行う)

## 準備

In [ ]:
# signateとの連携

!pip install signate

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

In [2]:
# コンペのデータをダウンロード
!signate download --competition-id={754}

sample_submit.csv

train.csv

test.csv


Download completed.


In [3]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 実装

In [4]:
%%capture
!pip install transformers datasets sentencepiece

### Import

In [6]:
from pathlib import Path
from multiprocessing import cpu_count
import random
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, f1_score

from datasets import load_dataset, Dataset, DatasetDict

from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from transformers import EvalPrediction
from transformers import set_seed

import torch
from torch import nn

### Config

In [8]:
EXP_NAME = 'exp01'
INPUT_DIR = Path('/content/drive/MyDrive/signate/mufj/Input')
OUTPUT_DIR = Path('/content/drive/MyDrive/signate/mufj/Output')
DEBUG = True
TEXT_COLUMNS = ['goal','country','duration','category1','category2','html_content']
MODEL_NAME = 'microsoft/deberta-v3-base'
N_SPLIT = 5
TRN_FOLDS = [0,1,2,3,4]
MAX_LEN = 512
SEED = 3090


training_args = TrainingArguments(
    output_dir = str(OUTPUT_DIR/EXP_NAME),
    seed = SEED,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs = 4,
    weight_decay = 4,
    fp16 = True,
    gradient_accumulation_steps=1,
    evaluation_strategy='epoch',
    save_total_limit = 1,
    save_strategy = 'epoch',
    metric_for_best_model = 'f1_score',
    load_best_model_at_end = True,
    greater_is_better=True

)

### Cross Validation

In [15]:
def create_folds(data, num_splits):
    data["kfold"] = -1

    mskf = StratifiedKFold(n_splits = num_splits, shuffle=True, random_state=42)
    data_labels = data['state']

    for f, (t_, v_) in enumerate(mskf.split(data, data_labels)):
        data.loc[v_, "kfold"] = f
    
    return data

data = pd.read_csv(INPUT_DIR / "train.csv")
data = create_folds(data, num_splits=N_SPLIT)
data.to_csv("train_folds.csv", index = False)
print("Folds created sucessfully")

Folds created sucessfully


### Preprocess

In [16]:
def text_to_input_ids(examples):
    return tokenizer(examples['text'] ,padding=False ,truncation=True, max_length=MAX_LEN)

def connect_text(df, text_cols, sep):
    text_df = df[text_cols].fillna('NAN').astype(str)
    connected_text = text_df[text_cols[0]].str.cat(text_df[text_cols[1:]], sep=sep)

    return connected_text

### Augumentation

expanding data

In [13]:
class RandomMask:
    def __init__(self, tokenizer, prob=0.5, mask_prob=0.15):
        self.mask_token_id = tokenizer.mask_token_id

ValueError: ignored